<a href="https://colab.research.google.com/github/CESARIUX2596/CS-Master/blob/master/Artificial_Vision_FInal_Proyect_Data_Augmentation_Aproach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guitar classification
In this project we will create a model that classifies guitar by body type in realtime using the videofeed of a camera.
The body types available to clasify are the following:


*   Acoustic
*   Double Cut
*   Les Paul
*   Telecaster
*   Strat
*   Ukulele


In this Aproach we will use data augmentation to provide more training information to the Convolutional Neuronal Network while training.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow import keras

In [6]:
# Mount Google drive as a device
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
'''
The folder structure contains the Train and Test directories
Each diractory has subfolders for each class that the classificatior will accept
'''
# Mount the folder with the dataset
os.chdir("/content/drive/My Drive/datasets/caos_guitar_dataset")
!ls

Test  Train
